# Titanic

Open project for [Network Tour of Data Science](https://github.com/mdeff/ntds_2017) EPFL class.

*Teo Stocco, Pierre-Alexandre Lee, Yves Lamonato, Charles Thiebaut*, EPFL.

nbviewer + binder

isolate code blocks in functions in separate Python module

project not shared

## Abstract

TODO

## 1 Introduction

Motivation

Icebergs and ships do not get well along each other. To avoid dramatic event
such as the one that happened a century ago, we aim at helping a noble quest:
differentiating icebergs and ships based on satellite imagery to see whether any
iceberg is drifting away and might cross the road of a ship.

Problem

Goal

Using what has been studied in class, we would extract features from the images
and then create a graph out of those. Getting inspiration from the following
paper: "Convolutional Neural Networks on Graphs with Fast Localized Spectral
Filtering" as well as in the following slides "Convolutional Neural Networks on
Graphs", we would then proceed to some analysis and binary classiffcation to
reach our goal. We aim at evaluating different ways we could extract features
and model them for later steps, especially ways of getting rotation invariance
on given data to avoid classical convolutional neural network limitation.

non euclidian

business

## 2 Data source

kaggle

### Description

angle

### Exploration

3D picture plotly

### Vizualization

PCA
clustering

### Cleaning


## 3 Preprocessing

set seed

### Scaling/normalization

### Features

### Validation set

## 4 Graphs

### Grid

circular

### 2D grid

### Pattern-based graph


## 5 Modelling

### Graph Convolution

fourier

chebyshev

### Pooling

graculus

korn?

algeibraic (spectral cut)

MST

pattern-based

## 6 Training

### Train/test split

### Skorch

## 7 Evaluation

### Models

### Stacking

## 8 Conclusion

### Further work

### Acknowledgements

## 9 References

- DEFFERRARD, Michaël, BRESSON, Xavier, et VANDERGHEYNST, Pierre. Convolutional neural networks on graphs with fast localized spectral filtering. In : Advances in Neural Information Processing Systems. 2016. p. 3844-3852.
- SHUMAN, David I., NARANG, Sunil K., FROSSARD, Pascal, et al. The emerging field of signal processing on graphs: Extending high-dimensional data analysis to networks and other irregular domains. IEEE Signal Processing Magazine, 2013, vol. 30, no 3, p. 83-98.
- BRONSTEIN, Michael M., BRUNA, Joan, LECUN, Yann, et al. Geometric deep learning: going beyond euclidean data. IEEE Signal Processing Magazine, 2017, vol. 34, no 4, p. 18-42.
